# Overview

PanGaia simplifies the access, exploration, and clustering analysis of the Gaia DR2 [catalogue](https://gea.esac.esa.int/archive/).
Its target audience is mainly the community of astromers who are *familiar* with the Gaia archive and that are interested in Star Formation.
This code closely follows the analysis described by [Canovas et al. 2019](https://ui.adsabs.harvard.edu/abs/2019A%26A...626A..80C/abstract) to identify more than 150 potential new candidates of the $\rho$-Ophiuchus Star Forming Region using Gaia astrometry.


### **PanGaia in a Nutshell:**
1. **Data Access**: An [ADQL](http://www.ivoa.net/documents/ADQL/2.0) cone-search (e.g. [link](https://gea.esac.esa.int/archive-help/adql/examples/index.html)) in the Gaia DR2 archive is performed using the [astroquery.gaia](https://astroquery.readthedocs.io/en/latest/gaia/gaia.html) package. Several extra columns are added to the queried table (like e.g. the distance, computed as the inverse of the parallax, $d = 1/\varpi$),
which is saved as a .vot table in the current directory. 


2. **Data Exploration:** The on-sky distribution of the data is visualised using [pyESASky](https://www.cosmos.esa.int/web/esdc/pyesasky), and the [plotly](https://plot.ly/python/) Python library is used for an interactive inspection of the sky position, proper motions, and distances of the data.


3. **Clustering Analysis:** Clusters (overdensities) in the 5-dimensional space defined by the 3 Cartesian dimensions (X, Y, and Z in Galactic coordinates), and the 2 proper motions $\mu_{\alpha}^*$ and $\mu_{\delta}$) of the downloaded dataset are identified using the 
[HDBSCAN](https://hdbscan.readthedocs.io/en/latest/) Machine Learning algorithm (an unsupervised density-based clustering algorithm). PanGaia automatically executes this algorithm for a range of *minimum Cluster Size* values. 


### **Products:**

The main outcomes of this notebooks are the  
1) Data sample downloaded from the Gaia archive, and the   
2) Clusters identified by the HDSBCAN algorithm.

The intermediate steps of the analysis are saved as .vot tables and .pdf figures.


### **Notes:**
* PanGaia has been written in a flexible way that allows the user to tune the code. For example, the visualization and clustering parts can easily be switched-off, which is advantageous if the user is only interested in performing a cone-search in the Gaia DR catalogue.
* Most of the functions and methods used by the code can run in an interactive way (default option, where the user is asked to input the needed parameters) or in a automatized way (examples are given accross this notebook).
* Most of the methods and functions used in this notebook contain several arguments that are not shown for the sake of simplicity. Expert python users interested in finding more about this are encouraged to take a look to the dedicated libraries saved in the "../library/" directory.

###### This notebook was written by Héctor Cánovas (hcanovas@sciops.esa.int), and is licensed under a MIT License

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1">Overview</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#PanGaia-in-a-Nutshell:" data-toc-modified-id="PanGaia-in-a-Nutshell:-1.0.1"><strong>PanGaia in a Nutshell:</strong></a></span></li><li><span><a href="#Products:" data-toc-modified-id="Products:-1.0.2"><strong>Products:</strong></a></span></li><li><span><a href="#Notes:" data-toc-modified-id="Notes:-1.0.3"><strong>Notes:</strong></a></span></li></ul></li></ul></li><li><span><a href="#Customize-Pangaia:" data-toc-modified-id="Customize-Pangaia:-2">Customize Pangaia:</a></span></li><li><span><a href="#1.--Load-Control-Sample" data-toc-modified-id="1.--Load-Control-Sample-3">1.- Load Control Sample</a></span></li><li><span><a href="#2.--Run-pyESASky" data-toc-modified-id="2.--Run-pyESASky-4">2.- Run pyESASky</a></span></li><li><span><a href="#3.--Construct-Data-Sample" data-toc-modified-id="3.--Construct-Data-Sample-5">3.- Construct Data Sample</a></span><ul class="toc-item"><li><span><a href="#3.1-Query-the-Gaia-Archive" data-toc-modified-id="3.1-Query-the-Gaia-Archive-5.1">3.1 Query the Gaia Archive</a></span></li><li><span><a href="#3.2-Load-dataset-to-Utils-package" data-toc-modified-id="3.2-Load-dataset-to-Utils-package-5.2">3.2 Load dataset to Utils package</a></span></li></ul></li><li><span><a href="#4.--Visual-Analysis" data-toc-modified-id="4.--Visual-Analysis-6">4.- Visual Analysis</a></span><ul class="toc-item"><li><span><a href="#4.1-Dynamic-data-exploration" data-toc-modified-id="4.1-Dynamic-data-exploration-6.1">4.1 Dynamic data exploration</a></span></li><li><span><a href="#4.2-On-Sky-distribution-exploration" data-toc-modified-id="4.2-On-Sky-distribution-exploration-6.2">4.2 On-Sky distribution exploration</a></span></li></ul></li><li><span><a href="#5.--Clustering-Analysis" data-toc-modified-id="5.--Clustering-Analysis-7">5.- Clustering Analysis</a></span><ul class="toc-item"><li><span><a href="#5.1-Preprocess-the-data" data-toc-modified-id="5.1-Preprocess-the-data-7.1">5.1 Preprocess the data</a></span></li><li><span><a href="#5.2-Apply-HDSBCAN" data-toc-modified-id="5.2-Apply-HDSBCAN-7.2">5.2 Apply HDSBCAN</a></span></li><li><span><a href="#5.3-Examine-the-HDBSCAN-outcomes" data-toc-modified-id="5.3-Examine-the-HDBSCAN-outcomes-7.3">5.3 Examine the HDBSCAN outcomes</a></span></li><li><span><a href="#5.4-Save-HDBSCAN-Output" data-toc-modified-id="5.4-Save-HDBSCAN-Output-7.4">5.4 Save HDBSCAN Output</a></span></li></ul></li></ul></div>

In [ ]:
# Python Basics ========================
import warnings, sys
warnings.filterwarnings('ignore', category=UserWarning)
sys.path.append('../library/')

# Custom ===============================
from lib_utils       import LibUtils       as Utils
from lib_gaia_query  import LibGaiaQuery   as GaiaQuery
from lib_interactive import LibInteractive as Interactive
from lib_cluster     import LibCluster     as Cluster
from lib_plotters    import LibPlotters    as Plotters

# pyESASky =============================
from pyesasky.pyesasky import ESASkyWidget

# Adjust cell size =====================
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>")) ## Adjust cell size as you wish

# Customize Pangaia:

The notebook is structured into 4 separated blocks or steps, each of which is indepent from the others. To decide how to run PanGaia set the steps below to ```True/False```:

In [ ]:
step_1 = True # Load Control Sample
step_2 = True # ESASky Plot
step_3 = True # Interactive/Dynamical Gaia Data exploration (Plotly)
step_4 = True # Clustering Analysis (HDSBCAN)

# 1.- Load Control Sample

The control sample is loaded if ```step_1 = True```. There are a number of variables that can be easily modified:
* Line  3: Set the control sample ```color``` and ```label```.
* Line  4: Set the directory from where the control samples are read. The code searches for the .vot tables stored in this directory and asks the user to choose one of them. Alternatively, the control sample can be specified as e.g., ```control.read_catalogue('../samples_control/sample_control_IC_348.vot')```
* Line  5: Set the number of control sample rows (```N_rows```) to be displayed (starting from the first row).
* Line 10: Change the features displayed in the plot & histogram by modifying the ```col_x, col_y, col_hist``` arguments.


In [ ]:
# Load Control Sample ==================================
if step_1:
    control = Utils(color = 'magenta', label = 'Sample Control')
    control.read_catalogue(sample_dir = '../samples_control/')
    control.display_N_rows(N_rows = 5, verbose = True)

    # Examine Velocity & Histograms ====================
    figures  = Plotters()
    figures.load_gaia_obj(control)
    figures.plot_2d_and_hist(col_x = 'pmra', col_y = 'pmdec', col_hist = 'distance')

# 2.- Run pyESASky
pyESASKy is a Python widget that opens an [ESASky](https://sky.esa.int/?target=273.2303166666667%2068.36347222222221&hips=DSS2%20color&fov=0.09993670884473815&cooframe=J2000&sci=true&lang=en) window locally. If 
```step_2 = True``` the widget overplots the control and/or queried sample on top of a RGB background mid-infrared image (by default). Different backgrounds and data-markers properties (e.g., symbols, colors, sizes) can be chosen using the interactive buttoms of pyESASky. A random image will be displayed if there is no control sample.

In [ ]:
# Run ESASKY ===========================================
if step_2:
    esasky = ESASkyWidget()
    display(esasky)
    
    if step_1:
        figures_0  = Plotters()
        figures_0.load_gaia_obj(control)
        figures_0.send_to_ESASky(esasky)    

# 3.- Construct Data Sample

## 3.1 Query the Gaia Archive

The next cell executes an [ADQL](http://www.ivoa.net/documents/ADQL/2.0) cone-search query in the Gaia archive. If there is a Control sample, its average Right Ascension and Declination ($\overline{\alpha_{\mathrm{ctrl}}}, \overline{\delta_{\mathrm{ctrl}}}$), maximum/minimum parallax ($\varpi_{\mathrm{ctrl - min/max}}$), and its projected radius on-sky ($r_{\mathrm{ctrl}}$) will be listed. The code will first ask for a name that will be used as the rootname for all the data products created by this notebook. Afterwards PanGaia will define the central coordinates of the query. If the control sample has been loaded then its $\overline{\alpha_{\mathrm{ctrl}}}, \overline{\delta_{\mathrm{ctrl}}}$ will be automatically used to center the cone search. Otherwise, the user will have to input the $\alpha, \delta$ (in $[^{\circ}]$) of the query. The code will additionally ask for the cone-search radius ($r_\mathrm{cone}$; in $[^{\circ}]$) and for the parallax range ($\varpi_{\mathrm{cone-min/max}}$; in [mas]) to constrain the query. To obtain a sample that encompases the entire control sample the query parameters should satisfy these conditions:  
$r_\mathrm{cone} > r_\mathrm{ctrl}$,  
$\varpi_\mathrm{cone-max} > \varpi_\mathrm{ctrl-max}$, and  
$\varpi_\mathrm{cone-min} < \varpi_\mathrm{ctrl-min}$.

The query can be authomatised by changing line 5 or 8 below including the necessary arguments as e.g., 

```sample_queried.set_cone_search_pars(sample_name = 'test', ra = 56.1, dec = 32.2, radii = 1.25, para_m = 2.4, para_M = 4.1)```

Additionally, PanGaia includes three quality constraints that can be easily adjusted in line 11 below:

```sample_queried.run_cone_search(quality_par_SN  = '10', quality_par_vis = '7',quality_par_ruwe = '1.40')```

The first argument of this method sets the minimum signal to noise (S/N) of the parallax of the sources downloaded. The second argument sets the minimum amount of [visibility periods](https://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html) used to compute the astrometric solution for each source, and the third argument sets the maximun value of the [RUWE](https://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_ruwe.html) parameter.

Once the coordinates and constraints are set the code executes the ADQL query using the [astroquery.gaia](https://astroquery.readthedocs.io/en/latest/gaia/gaia.html) package, downloading a .vot table with the query output to the current directory. Information regarding the query, its status, and the general properties of the downloaded sample are displayed.

In [ ]:
# Set Cone-search query parameters and constraints =====
sample_queried = GaiaQuery()
if step_1:
    control.print_cone_properties()
    sample_queried.set_cone_search_pars(ra = control.ADQL['ra'].value, dec = control.ADQL['dec'].value)
else:
    sample_queried.set_cone_search_pars()

# Execute cone-search query and download data ==========
sample_queried.run_cone_search(quality_par_SN  = '10.0', quality_par_vis = '7.0', quality_par_ruwe = '1.40')

## 3.2 Load dataset to Utils package
Once the query is finished the user can set the color and label associated to this dataset by modifying the ```color``` and ```label``` arguments set in line 2. By defaul, the label is the name set at the beggining of the query in the previous cell and the color is set to grey (except for visualisation with pyESASky, where this sample is represented as white squares by default as indicated in line 11).

In [ ]:
# Load data sample for analysis ======================== 
sample_gaia = Utils(color = 'grey', label = sample_queried.sample_name)
sample_gaia.read_catalogue(sample_queried.cat, save_sample = True, print_vrad = True)
sample_gaia.display_N_rows(N_rows = 5, verbose = True)


# Show output in ESASky ================================
if step_2:
    figures  = Plotters()
    figures.load_gaia_obj(sample_gaia)
    figures.send_to_ESASky(esasky, color= 'white', radius = sample_gaia.ADQL['radii'].value)

# 4.- Visual Analysis

The next two cells will only operate if ```step_3 = True```.

## 4.1 Dynamic data exploration
The next cell allows to inspect the dataset using the [plotly](https://plot.ly/python/) Python library. This tool is useful to identify overdensities (clusters!) hidden in the dataset. PanGaia opens 3 interactive panels, each of them having a dedicated menu to explore the displayed dimensions - to access it simply move the cursor around the top of each panel. A powerful utility of this menu is the selection tool that allows to select a subsample of points using the cursor. This selection can be erased by clicking on the "clear" button of the widget, and there is no limit in the amount of times that this selection can be repeated. By default, the control sample (if any) is represented as magenta circles. The user can change the colors of the downloaded and selected samples using the  and ```color_high``` arguments in line 3. By default, those are set to semi-transparent black (to help to visualise the overdensities) and to cyan, respectively. The selected datapoints (if any), will be highlighted in all the panels simultaneously.

In [ ]:
# Examine astrometry ===================================
if step_3:
    i_figures = Interactive(color_all = 'black',color_high = 'cyan')
    i_figures.load_gaia_cat(sample_gaia.cat)
    if step_1:
        i_figures.load_control_obj(control)
    i_figures.explore_and_select(opacity = 0.5, border_width = 1)
    display(i_figures.dashboard)

## 4.2 On-Sky distribution exploration
The cell below sends the previously selected subsample (if any) to the pyESASky widget. This subsample can be accessed throught the ```i_figures.selected.cat``` attribute set by the ```i_figures.export_selected()``` method (line 4 below). Additionally, the 3D spatial distribution in Cartesian-Galactic coordinates of the downloaded, control (if any) and selected (if any) datasets is displayed in a dynamic panel. The size of the circular markers representing these samples can be adjusted by changing the ```size``` argument in line 11.

In [ ]:
# Show selected sample (if any) in ESASky ==============
if step_2 and step_3 and hasattr(i_figures, 'cat_subsamp'):
    print('Plotting Selected sample in ESASky')
    i_figures.export_selected()
    figures_1  = Plotters()
    figures_1.load_gaia_obj(i_figures.selected)
    figures_1.send_to_ESASky(esasky)

# Show 3D distribution =================================
if step_3:
    i_figures.show_3D_space(size = 5)
    display(i_figures.fig_3D)

# 5.- Clustering Analysis
The next three cells will only operate if ```step_4 = True```.


## 5.1 Preprocess the data

The downloaded dataset is *standarised* (or scaled) with the [preprocessing](https://scikit-learn.org/stable/modules/preprocessing.html) package of the [scikit-learn](https://scikit-learn.org/stable/index.html) Python library. By default the standarization (and posterior clustering analysis) is applied over the 3 spatial dimensions in Galactic coordinates and the 2 proper motions ($X_\mathrm{Gal}, Y_\mathrm{Gal}, Z_\mathrm{Gal}, \mu_{\alpha}^*, \mu_{\delta}$). Three different scaling methods ('standard' (default), 'minmax', and 'robust') are currently implemented in PanGaia. Both the scaling features and method can be set by changing the ```scl_features``` and ```scaler```arguments of the ```data_clustering``` Class in line 4 below. Once the data is scaled it be saved as a .vot table and its histogram distributions will be displayed. 

In [ ]:
# Data Scaling =========================================
if step_4:
    data_clustering = Cluster()
    data_clustering.load_gaia_obj(sample_gaia, scl_features = ['X_gal', 'Y_gal', 'Z_gal', 'pmdec', 'pmra'], 
                                  scaler = 'standard')

## 5.2 Apply HDSBCAN
After scaling the data PanGaia applies the [HDBSCAN](https://hdbscan.readthedocs.io/en/latest/#) algorithm to identify clusters within the analised dataset. The main *hyperparameters* of HDBSCAN are the *minimum Cluster Size* and the *minimum samples*. The algorithm is executed  through a range of minimum Cluster Size values to show the HDSBCAN behavior for different values of this hyperparameter. By default, the user will be asked to input the minimum samples value and a probability thresold. The identified clusters will only contain sources whose membership probability is higher than this thresold. Both the probability threshold and the minimum samples  can be set by changing line 2 below to e.g.  ```data_clustering.run_multi_hdbscan(probability = 0.8, min_samples = 1)```. The amount of clusters found and their size will be printed for each execution of the algorithm. Finally, a color-coded bar chart presenting the amount of clusters and their size as a function of minimum Cluster Size will be displayed.

In [ ]:
# Apply HDSBCAN ========================================
if step_4:
    data_clustering.run_multi_hdbscan(probability = None, min_samples = -1)
    data_clustering.plot_barchart()

## 5.3 Examine the HDBSCAN outcomes
The number and sizes of the clusters found by HDSBCAN depend on the properties of the analysed dataset as well as the *minimum Cluster Size* and *minimum samples* hyperparameters, as illustrated by the bar chart diagram displayed above. The next cell allows to examine the individual outputs of HDBSCAN as a function of the minimum Cluster Size, and compare them to the control sample if ```step_1 = True```. If there is a control sample, its distance distribution will be shown as a transparent bar-filled histogram on top of the distance distribution sof the found clusters. We recommend to examine the different algorithm outcomes before proceeding saving the results in the next (and last) cell.

In [ ]:
# Show selected HDBSCAN results ========================
if step_4:
    data_clustering.run_hdbscan(probability = data_clustering.probability, min_samples = data_clustering.min_samples, verbose = True)
    if step_1:
        data_clustering.load_control_obj(control)
        data_clustering.compare_to_control()
    data_clustering.plot_clusters()

## 5.4 Save HDBSCAN Output

In its final step, PanGaia saves the clusters found by HDSBCAN as a .vot table, and plot them in pyESASky (if ```step_2 = True```)

In [ ]:
# Final Stage: Plot choosen clusters & save them
if step_2:
    data_clustering.send_to_ESASky(esasky, radius = sample_gaia.ADQL['radii'].value)
if step_4:
    data_clustering.save_clusters()